<h1>Simulating the action potential with random walk of ions</h1>

<h3>GroupXX: Us :p</h3>

<h2>Abstract</h2>

<h2>1 - Theory</h2>

**Teori**

<h2>2 - Excercises</h2>

**Excercises**

<h2>3 - Programming</h2>


## 2 - The diffusion equation

### Exercise 2.1
Show that the function

$$ 
\tilde{\phi}(x,t) = \frac{1}{\sqrt{4 \pi Dt}} e^{-\frac{(x-\mu)^2}{4Dt}},
$$

where $D$ is a constant, is a solution of the diffusion equation

$$
\frac{\partial \phi (x,t)}{\partial t} = D \frac{\partial^2 \phi (x,t)}{\partial x^2}.
$$

The left side of the equation is solved by differentiating with respect to $t$.

$$
\frac{\partial}{\partial t} \frac{1}{\sqrt{4 \pi Dt}} e^{-\frac{(x-\mu)^2}{4Dt}} 
= 
\left( \frac{1}{\sqrt{4 \pi Dt}} \right)' e^{-\frac{(x-\mu)^2}{4Dt}} 
+ 
\frac{1}{\sqrt{4 \pi Dt}} \left(e^{-\frac{(x-\mu)^2}{4Dt}}\right)' 
=
- \frac{1}{4\sqrt{D\pi}t^{3/2}}e^{-\frac{(x-\mu)^2}{4Dt}} 
+
\frac{(x-\mu)^2}{(4D)^{3/2}\sqrt{\pi}t^{5/2}}e^{-\frac{(x-\mu)^2}{4Dt}}
=
\frac{1}{4\sqrt{D\pi}t^{3/2}} e^{-\frac{(x-\mu)^2}{4Dt}}
\left(\frac{(x-\mu)^2}{\sqrt{4}tD}-1\right)
$$

The right side is then solved by differentiating twice with respect to $x$, times the constant $D$.

$$
\frac{\partial}{\partial x} \frac{1}{\sqrt{4 \pi Dt}} e^{-\frac{(x-\mu)^2}{4Dt}} = - \frac{2(x-\mu)}{(4Dt)^{3/2}\sqrt{\pi}}e^{-\frac{(x-\mu)^2}{4Dt}}
$$
<br>
$$
- \frac{\partial}{\partial x} \frac{2(x-\mu)}{(4Dt)^{3/2} \sqrt{\pi}} e^{-\frac{(x-\mu)^2}{4Dt}}
= \frac{2(x-\mu)}{(4Dt)^{3/2}\sqrt{\pi}} \left( e^{-\frac{(x-\mu)^2}{4Dt}} \right)' 
- \left( \frac{2(x-\mu)}{(4Dt)^{3/2}\sqrt{\pi}}\right)' e^{-\frac{(x-\mu)^2}{4Dt}}
= \frac{4(x-\mu)^2}{(4Dt)^{5/2}\sqrt{\pi}} e^{-\frac{(x-\mu)^2}{4Dt}} 
- \frac{2(x-\mu)}{(4Dt)^{3/2}\sqrt{\pi}} e^{-\frac{(x-\mu)^2}{4Dt}}
= \frac{1}{4\sqrt{\pi}(Dt)^{3/2}} e^{-\frac{(x-\mu)^2}{4Dt}}
\left(\frac{(x-\mu)^2}{\sqrt{4}tD}-1\right)
$$
<br>
$$
D \frac{\partial^2 \tilde{\phi}(x,t)}{\partial x^2} = \frac{1}{4\sqrt{D\pi}t^{3/2}} e^{-\frac{(x-\mu)^2}{4Dt}}
\left(\frac{(x-\mu)^2}{\sqrt{4}tD}-1\right)
$$

The function $\tilde{\phi}(x,t)$ is therefore a solution of the diffusion equation.

### Exercise 2.2
**2.2.1**

Suppose we have an infinite 1D system with constant diffusion coefficient D. With the particle distribution

$$
\phi (x, 0) = \delta (x-x_0),
$$

where $\delta (x)$ is the Dirac delta function, the time derivative can be found by using the diffusion equation. Firstly the Dirac delta function can also be written as

$$
\phi (x, 0) =  \delta (x-x_0) = \frac{1}{2\pi} \int_{\infty}^{\infty} dk e^{-ik(x-x_0)}.
$$

Deriving twice with respects to $x$ and multiplying with $D$ gives the time derivative of $\phi(x, t)$.

$$
\frac{\partial \phi(x, t)}{\partial t} = D \frac{\partial^2 \phi (x,t)}{\partial x^2} = - \frac{Dk^2}{2\pi} \int_{\infty}^{\infty} dk e^{-ik(x-x_0)}
$$

The Taylor expansion of $\phi(x, t)$ is

$$
\phi(x, t) = \phi (x, 0) + \sum_{n=1}^{\infty} \frac{t^n}{n!} \frac{\partial^n \phi(x,t')}{\partial t'^n} \Bigg|_{t'=0}.
$$

To expand the sum, the previous process can be used to further derivate $\phi(x, t)$

$$
\phi(x, t) = \frac{1}{2\pi} \int_{\infty}^{\infty} dk e^{-ik(x-x_0)} 
- \frac{Dk^2t}{2\pi} \int_{\infty}^{\infty} dk e^{-ik(x-x_0)} 
$$
<br>
$$
+ \frac{Dk^4t^2}{2\pi2!}\int_{\infty}^{\infty}dke^{-ik(x-x_0)}
-\cdots.
$$

Every element of the sum is thereby given as
$$
(-1)^n \frac{Dk^{2n}t^n}{2\pi n!}\int_{\infty}^{\infty}dke^{-ik(x-x_0)}, \quad n=1,2,3...\infty.
$$

This pattern makes it possible to write

$$
\phi (x,t) = \frac{1}{2\pi} \int_{\infty}^{\infty}dke^{-ik(x-x_0)} \left( 1 - Dk^2t + \frac{Dk^4t^2}{2!} - \frac{Dk^6t^3}{3!} \pm\cdots \right).
$$

The expression inside the paracentese is resembles the Maclaurin series for $e^x$

$$
e^{-Dk^2t} = 1 - Dk^2t + Dk^4t^2 - Dk^6t^3\cdots.
$$

The particle distribution is now neatly expressed as
$$
\phi(x, t) = \frac{1}{2\pi}\int_{\infty}^{\infty}dk e^{-Dk^2t} e^{-ik(x-x_0)}.
$$

In [1]:
import numpy as np
from matplotlib import pyplot as plt 
from scipy.stats import norm

N = 1000
steps = 100 
field = 2*steps
x_vec = np.linspace(-steps, steps, field)

particles = np.zeros([N, field])
particles.T[steps] = np.ones(N)

for i in range(steps):
    steps_vec = np.around(np.random.rand(N))
    steps_vec[steps_vec == 0] = -1
    for j in range(N):
        particles[j] = np.roll(particles[j], int(steps_vec[j]))

dist = np.sum(particles, 0)
params = norm.fit(dist)
print(params)
fitted_pdf = norm.pdf(x_vec, loc = params[0], scale = params[1])

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.bar(x_vec, dist)
ax2.plot(x_vec, fitted_pdf)

plt.show()

(5.0, 16.090680532531866)


<Figure size 640x480 with 2 Axes>